In [1]:
import csv
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional
from sklearn import preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.model_selection import train_test_split

In [25]:
df = pd.read_csv('./data/imdb.csv',delimiter=',', header=None)

In [26]:
df = df.sample(frac=1)
X = df[0]
Y = df[1]

In [27]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
if Y.dtype == 'int64':
    Y = np.array(Y, dtype='str')

[nltk_data] Downloading package stopwords to /Users/chris/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [29]:
vocab_size = 5000 # make the top list of words (common words)
embedding_dim = 64
max_length = 200
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>' # OOV = Out of Vocabulary
training_portion = .8
units = 1024

In [30]:

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [31]:
train_sequences = tokenizer.texts_to_sequences(X_train)

In [32]:
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [33]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

validation_sequences = tokenizer.texts_to_sequences(X_test)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [34]:
labels = set()
for l in Y_train:
    labels.add(l)

In [35]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(Y_train))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(Y_test))

In [36]:
from keras.layers import Dense, Embedding, Dropout, GRU
from keras.models import Sequential
from keras import layers


model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(0.5))
model.add(GRU(embedding_dim))
model.add(layers.Dense(5, activation='sigmoid'))
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)



In [ ]:
num_epochs = 10
history = model.fit(train_padded, training_label_seq, epochs=num_epochs,verbose=2)

Epoch 1/10
1329/1329 - 157s - loss: 0.6033 - accuracy: 0.6193
Epoch 2/10
1329/1329 - 148s - loss: 0.2843 - accuracy: 0.8825
Epoch 3/10
1329/1329 - 150s - loss: 0.2396 - accuracy: 0.9032
Epoch 4/10
1329/1329 - 139s - loss: 0.2129 - accuracy: 0.9160
Epoch 5/10
1329/1329 - 140s - loss: 0.1946 - accuracy: 0.9238
Epoch 6/10
1329/1329 - 126s - loss: 0.1777 - accuracy: 0.9330
Epoch 7/10
1329/1329 - 76s - loss: 0.1625 - accuracy: 0.9386
Epoch 8/10
1329/1329 - 76s - loss: 0.1497 - accuracy: 0.9436
Epoch 9/10
1329/1329 - 77s - loss: 0.1379 - accuracy: 0.9487
Epoch 10/10


In [22]:
pred = model.predict(validation_padded)
pred = pred.argmax(axis=-1)
pred

array([2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       1, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2,
       2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2,
       2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1,

In [23]:
test = []
for i in validation_label_seq:
    test.append(i[0])
test = np.asarray(test)

In [24]:
metrics.accuracy_score(pred, test)

0.9808612440191388